In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import os, sys
import urllib3
import time
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("."), '..')))
from DataBase.DBUtilities import BRVMDatabase, DataPreProcessor

In [2]:
kl_db = BRVMDatabase("KL.db")
kl_db.update()

✅ Table 'stock_prices' successfully  created.
✅ Table 'indices_performances' successfully  created.
✅ Table 'market_cap' successfully  created.
✅ Table 'per_data' successfully  created.
✅ Table 'fundamental_data' successfully  created.
=====================Before daily update:=============================

Table status (rows):
stock_prices : 45 rows of ['Date', 'Ticker', 'Description', 'Open', 'High', 'Low', 'Close', 'Volume']
indices_performances : 0 rows of ['Date', 'Ticker', 'Description', 'Open', 'High', 'Low', 'Close', 'Volume']
market_cap : 49 rows of ['Ticker', 'Date', 'stock_description', 'stock_number', 'floting_cap', 'globale_cap', 'globale_cap_pct', 'trade_number', 'trade_value', 'globale_trade_value_pct', 'PER']
per_data : 0 rows of ['Ticker', 'Stock_Description', 'Nombre_titres_echanges', 'Valeur_echangee', 'PER', 'Valeur_globale_echangee_pct', 'Date']
fundamental_data : 0 rows of ['Ticker', 'Volume', 'Next', 'Market', 'Revenue', 'Average_3m', 'EPS', 'P_E', 'Beta', 'Divide

In [46]:
tables = kl_db.get_all_tables()

In [50]:
text = ["Ticker", "Date", "Description"]
for t in tables:
    print(t, [(e, "REAL") if not e in text else (e, "TEXT NOT NULL") for e in kl_db._get_table_columns(t)])

per_data [('Ticker', 'TEXT NOT NULL'), ('Stock_Description', 'REAL'), ('Nombre_titres_echanges', 'REAL'), ('Valeur_echangee', 'REAL'), ('PER', 'REAL'), ('Valeur_globale_echangee_pct', 'REAL'), ('Date', 'TEXT NOT NULL')]
stock_prices [('Date', 'TEXT NOT NULL'), ('Ticker', 'TEXT NOT NULL'), ('Description', 'TEXT NOT NULL'), ('Open', 'REAL'), ('High', 'REAL'), ('Low', 'REAL'), ('Close', 'REAL'), ('Volume', 'REAL')]
indices_performances [('Date', 'TEXT NOT NULL'), ('Ticker', 'TEXT NOT NULL'), ('Description', 'TEXT NOT NULL'), ('Open', 'REAL'), ('High', 'REAL'), ('Low', 'REAL'), ('Close', 'REAL'), ('Volume', 'REAL')]
fundamental_data [('Ticker', 'TEXT NOT NULL'), ('Volume', 'REAL'), ('Next Earnings Date', 'REAL'), ('Market Cap', 'REAL'), ('Revenue', 'REAL'), ('Average 3m Volume', 'REAL'), ('EPS', 'REAL'), ('P/E Ratio', 'REAL'), ('Beta', 'REAL'), ('Dividend', 'REAL'), ('Yield', 'REAL'), ('Daily Trend', 'REAL'), ('Weekly Trend', 'REAL'), ('Monthly Trend', 'REAL'), ('1 Day perf', 'REAL'), ('1 

In [41]:
print(u'\N{check}')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-8: unknown Unicode character name (1902552421.py, line 1)

In [23]:
def insert_daily_market_cap(self):
    ldf = self.dp.get_today_data_from_brvm()
    col = ['Date', 'Stock Description', 'Nombre de titres', 'Capitalisation flottante', 'Capitalisation globale',
           'Capitalisation globale (%)', 'Nombre de titres échangés', 'Valeur échangée', 'valeur globale échangée %', 'PER']
    df_m = ldf[1].merge(ldf[2], on=["Stock Description", "Date", "Ticker"])[col]
    df_m.columns = ['Date', 'stock_description', 'stock_number', 'floting_cap', 'globale_cap', 'globale_cap_pct', 'trade_number', 'trade_value', 'globale_trade_value_pct', 'PER']
    df_m["Date"] = self.dp,make_date(df_m['Date'])
    self.insert_dataframe(df_m, "market_cap")
    

,Date,stock_description,stock_number,floting_cap,globale_cap,globale_cap_pct,trade_number,trade_value,globale_trade_value%,PER
Ticker,,,,,,,,,,
ABJC,2025-06-13,SERVAIRABIDJANCOTED'IVOIRE,1.091200e+07,4.015579e+09,2.007808e+10,0.17,3198.0,5625880.0,0.29,13.25
BICB,2025-06-13,BANQUEINTERNATIONALEPOURL’INDUSTRIEETLECOMMERC...,5.775980e+07,0.000000e+00,3.231661e+11,2.74,1143.0,6341095.0,0.32,11.76
BICC,2025-06-13,BICICOTED'IVOIRE,1.666667e+07,8.724365e+10,2.683334e+11,2.27,920.0,15578800.0,0.79,10.23
BNBC,2025-06-13,BERNABECOTED'IVOIRE,6.624000e+06,2.636911e+09,7.551360e+09,0.06,111.0,126540.0,0.01,1032.53
BOAB,2025-06-13,BANKOFAFRICABENIN,4.056105e+07,4.191522e+10,1.849584e+11,1.57,4584.0,20811885.0,1.06,9.43
BOABF,2025-06-13,BANKOFAFRICABURKINAFASO,4.400000e+07,2.330648e+10,1.601600e+11,1.36,12822.0,48436320.0,2.46,7.14
BOAC,2025-06-13,BANKOFAFRICACOTED'IVOIRE,4.000000e+07,3.370518e+10,2.402000e+11,2.03,14141.0,85349685.0,4.34,7.61
BOAM,2025-06-13,BANKOFAFRICAMALI,2.745000e+07,1.008933e+10,8.235000e+10,0.70,33200.0,101302395.0,5.15,9.16
BOAN,2025-06-13,BANKOFAFRICANIGER,2.080000e+07,1.422349e+10,5.595200e+10,0.47,2031.0,5413075.0,0.28,11.23


In [36]:
p = kl_db.get_prices(["BNBC", "ABJC"], "2025-01-30")#, "2025-05-30")
p
#p_m = kl_db.get_close_matrix(["BNBC", "ABJC"], "2025-03-30")#, "2025-04-30")

,Date,Ticker,Description,Open,High,Low,Close,Volume
0,2025-06-13 00:00:00,ABJC,SERVAIR ABIDJAN CI,1730.0,1840.0,1730.0,1840.0,3198.0
1,2025-06-13 00:00:00,BNBC,BERNABE,1140.0,1140.0,1140.0,1140.0,111.0
2,2025-06-12 00:00:00,BNBC,BERNABE,1140.0,1140.0,1140.0,1140.0,356.0
3,2025-06-12 00:00:00,ABJC,SERVAIR ABIDJAN CI,1600.0,1720.0,1600.0,1720.0,1706.0
4,2025-06-11 00:00:00,BNBC,BERNABE,1150.0,1150.0,1065.0,1065.0,0.0
...,...,...,...,...,...,...,...,...
185,2025-02-03 00:00:00,ABJC,SERVAIR ABIDJAN CI,1835.0,1835.0,1835.0,1835.0,0.0
186,2025-02-03 00:00:00,BNBC,BERNABE CI,1035.0,1035.0,1025.0,1025.0,0.0
187,2025-01-31 00:00:00,BNBC,BERNABE CI,1040.0,1040.0,1035.0,1035.0,0.0
188,2025-01-30 00:00:00,ABJC,SERVAIR ABIDJAN CI,1830.0,1835.0,1830.0,1835.0,0.0


In [4]:
kl_db.execute_query("SELECT * FROM indices_performances WHERE TICKER = 'BRVMAG' ORDER BY Date DESC")

,Date,Ticker,Description,Open,High,Low,Close,Volume
0,2025-06-12 00:00:00,BRVMAG,BRVM - AGRICULTURE,298.34,298.34,298.34,298.34,0.0
1,2025-06-11 00:00:00,BRVMAG,BRVM - AGRICULTURE,294.74,294.74,294.74,294.74,0.0
2,2025-06-11 00:00:00,BRVMAG,BRVM - AGRICULTURE,294.74,294.74,294.74,294.74,0.0
3,2025-06-06 00:00:00,BRVMAG,BRVM-AGRICULTURE,283.04,283.04,283.04,283.04,0.0
4,2025-06-05 00:00:00,BRVMAG,BRVM - AGRICULTURE,283.04,283.04,283.04,283.04,0.0
...,...,...,...,...,...,...,...,...
64,2025-03-13 00:00:00,BRVMAG,BRVM - AGRICULTURE,221.5,221.5,221.5,221.5,0.0
65,2025-03-12 00:00:00,BRVMAG,BRVM - AGRICULTURE,224.79,224.79,224.79,224.79,0.0
66,2025-03-11 00:00:00,BRVMAG,BRVM - AGRICULTURE,224.24,224.24,224.24,224.24,0.0
67,2025-03-10 00:00:00,BRVMAG,BRVM - AGRICULTURE,224.73,224.73,224.73,224.73,0.0


In [6]:
mde = MarketData(p)

In [7]:
mde.get_ma().dropna().head()

Ticker,ABJC,BNBC
Date,,
2025-02-28,1844.00,1044.25
2025-03-03,1846.00,1048.50
2025-03-04,1845.50,1052.50
2025-03-05,1845.00,1057.00
2025-03-06,1846.75,1061.00


In [8]:
mde.normalize()

Open      High       Low     Close    Volume
Date       Ticker                                                  
2025-05-30 ABJC    0.530380  0.503766  0.562303  0.535397  0.366393
           BNBC   -0.700750 -0.735226 -0.815496 -0.849797 -0.485431
2025-05-29 ABJC    0.476265  0.503766  0.508271  0.535397  0.139382
           BNBC   -0.700750 -0.735226 -0.666910 -0.700414 -0.453061
2025-05-28 ABJC    0.476265  0.503766  0.508271  0.535397 -0.059519
...                     ...       ...       ...       ...       ...
2025-02-03 ABJC    1.166239  1.143685  1.197171  1.173672 -0.485431
           BNBC   -0.998386 -1.034763 -0.991098 -1.026342 -0.485431
2025-01-31 BNBC   -0.984857 -1.021147 -0.964082 -0.999181 -0.485431
2025-01-30 ABJC    1.152710  1.143685  1.183663  1.173672 -0.485431
           BNBC   -0.957800 -0.993917 -0.950574 -0.985601 -0.485431

[172 rows x 5 columns]

In [37]:
kl_db.get_per(["ABJC"])

DatabaseError: Execution failed on sql 'SELECT Date,Ticker,per FROM per_data WHERE Ticker ='['ABJC']' ORDER BY Date DESC': near "ABJC": syntax error

In [90]:
perb = kl_db.execute_query("SELECT * FROM per_data")
perb["Date"] = kl_db.dp.make_date(kl_db.dp.make_date(perb["Date"]))
perb

,Ticker,Stock_Description,Nombre_titres_echanges,Valeur_echangee,PER,Valeur_globale_echangee_pct,Date
0,ABJC,SERVAIR ABIDJAN COTE D'IVOIRE,6087.0,9738890.0,11.96,1.52,2025-06-09
1,BICB,BANQUE INTERNATIONALE POUR L’INDUSTRIE ET LE C...,2156.0,11527840.0,11.31,1.80,2025-06-09
2,BICC,BICI COTE D'IVOIRE,211.0,3378695.0,10.74,0.53,2025-06-09
3,BNBC,BERNABE COTE D'IVOIRE,1003.0,1138335.0,1028.00,0.18,2025-06-09
4,BOAB,BANK OF AFRICA BENIN,792.0,3532115.0,9.27,0.55,2025-06-09
...,...,...,...,...,...,...,...
93,TTLC,TOTALENERGIES MARKETING COTE D'IVOIRE,4189.0,12160165.0,19.38,1.90,2025-06-06
94,TTLS,TOTALENERGIES MARKETING SENEGAL,1260.0,3410770.0,12.71,0.53,2025-06-06
95,TTRC,TRITURAF Ste en Liquid,0.0,0.0,0.00,0.00,2025-06-06
96,UNLC,UNILEVER COTE D'IVOIRE,9.0,104400.0,166.36,0.02,2025-06-06


In [19]:
stock_per

,Date,Ticker,PER
0,2025-06-12 00:00:00,ABJC,12.32
1,2025-06-12 00:00:00,BICB,11.76
2,2025-06-12 00:00:00,BICC,10.80
3,2025-06-12 00:00:00,BNBC,1032.53
4,2025-06-12 00:00:00,BOAB,9.43
...,...,...,...
191,2025-06-06 00:00:00,TTLC,19.38
192,2025-06-06 00:00:00,TTLS,12.71
193,2025-06-06 00:00:00,TTRC,0.00
194,2025-06-06 00:00:00,UNLC,166.36


In [20]:
stock_fd

,Ticker,Volume,Next Earnings Date,Market Cap,Revenue,Average 3m Volume,EPS,P/E Ratio,Beta,Dividend,...,Monthly Trend,1 Day perf,1 Week perf,1 Month perf,YTD,1 Year perf,3 Years perf,Date,PER,Capitalisation_flottante
0,ABJC,"6,087",--,"18,110,000,000","12,470,000,000","2,185",138.84,11.52,0.16,206.2,...,-1.0,0.00%,3.75%,3.43%,-12.63%,-16.16%,5.06%,2025-06-05,12.32,1000000


In [21]:
stock_per = kl_db.get_per()
#stock_per.index = stock_per["Ticker"]
#stock_fd.index = stock_fd["Ticker"]
stock_per.merge(stock_fd, on=["Ticker"])

,Date_x,Ticker,PER_x,Volume,Next Earnings Date,Market Cap,Revenue,Average 3m Volume,EPS,P/E Ratio,...,Monthly Trend,1 Day perf,1 Week perf,1 Month perf,YTD,1 Year perf,3 Years perf,Date_y,PER_y,Capitalisation_flottante
0,2025-06-12 00:00:00,ABJC,12.32,"6,087",--,"18,110,000,000","12,470,000,000","2,185",138.84,11.52,...,-1.0,0.00%,3.75%,3.43%,-12.63%,-16.16%,5.06%,2025-06-05,12.32,1000000
1,2025-06-11 00:00:00,ABJC,11.52,"6,087",--,"18,110,000,000","12,470,000,000","2,185",138.84,11.52,...,-1.0,0.00%,3.75%,3.43%,-12.63%,-16.16%,5.06%,2025-06-05,12.32,1000000
2,2025-06-09 00:00:00,ABJC,11.96,"6,087",--,"18,110,000,000","12,470,000,000","2,185",138.84,11.52,...,-1.0,0.00%,3.75%,3.43%,-12.63%,-16.16%,5.06%,2025-06-05,12.32,1000000
3,2025-06-06 00:00:00,ABJC,11.96,"6,087",--,"18,110,000,000","12,470,000,000","2,185",138.84,11.52,...,-1.0,0.00%,3.75%,3.43%,-12.63%,-16.16%,5.06%,2025-06-05,12.32,1000000


In [22]:
stock_prices = kl_db.get_prices(["ABJC"], "2020-01-01") #, "2025-05-30")
stock_fd = get_fundamental("ABJC")
stock_fd["PER"], stock_fd["Capitalisation_flottante"] = kl_db.get_per("ABJC")["PER"][0], 1000000
stock = Stock("ABJC", stock_prices, stock_fd)
ste = StockEvaluator()
ste.compute_score(stock)

150377.84933802206

In [28]:
type('')

str

In [34]:
def get_fundamental(ticker):
    ticker_list = [ticker] if type(ticker) is str else ticker
    self = kl_db
    tls = ','.join([f"'{t}'" for t in ticker_list])
    df = self.execute_query(f"SELECT * FROM fundamental_data WHERE Ticker in ({tls})")
    return df
get_fundamental(["ABJC", "BNBC"])

,Ticker,Volume,Next Earnings Date,Market Cap,Revenue,Average 3m Volume,EPS,P/E Ratio,Beta,Dividend,...,Daily Trend,Weekly Trend,Monthly Trend,1 Day perf,1 Week perf,1 Month perf,YTD,1 Year perf,3 Years perf,Date
0,ABJC,"6,087",--,"18,110,000,000","12,470,000,000","2,185",138.84,11.52,0.16,206.2,...,0.0,-1.0,-1.0,0.00%,3.75%,3.43%,-12.63%,-16.16%,5.06%,2025-06-05
1,BNBC,"1,003",--,"7,520,000,000","46,740,000,000","1,718",-,-,-0.27,-,...,0.0,1.0,-1.0,7.08%,3.18%,13.50%,6.57%,13.50%,-49.33%,2025-06-05


In [9]:
import pandas as pd

class StockEvaluator:
    def __init__(self, criteria=None):
        self.criteria = criteria or {
            "momentum": 0.3,
            "relative_price": 0.2,
            "volatility": 0.1,
            "volume": 0.15,
            "per": 0.1,
            "cap_flo": 0.15
        }

    def compute_score(self, stock):
        score = 0
        for key, weight in self.criteria.items():
            val = stock.get_indicator(key)
            if val is not None:
                score += weight * val
        return score

    def rank_stocks(self, stock_list):
        scored = [(stock, self.compute_score(stock)) for stock in stock_list]
        return sorted(scored, key=lambda x: x[1], reverse=True)

    def build_stocks_from_data(self, price_df: pd.DataFrame, fundamental_df: pd.DataFrame):
        grouped = price_df.groupby("Ticker")
        stocks = []
        for ticker, df in grouped:
            fund_row = fundamental_df[fundamental_df['Ticker'] == ticker]
            stock = Stock(ticker, df.copy(), fund_row)
            stocks.append(stock)
        return stocks


class Stock:
    def __init__(self, ticker, price_df: pd.DataFrame, fund_row: pd.DataFrame):
        self.ticker = ticker
        self.price_df = price_df.sort_values("Date")
        self.indicators = {}
        self._compute_indicators(fund_row)

    def _compute_indicators(self, fund_row):
        if len(self.price_df) >= 21:
            self.indicators["momentum"] = self.price_df["Close"].iloc[-1] / self.price_df["Close"].iloc[-21] - 1
        if len(self.price_df) >= 5:
            self.indicators["volatility"] = self.price_df["Close"].pct_change().rolling(5).std().iloc[-1]
            self.indicators["volume"] = self.price_df["Volume"].rolling(5).mean().iloc[-1]
        if len(self.price_df) >= 30:
            self.indicators["relative_price"] = self.price_df["Close"].iloc[-1] / self.price_df["Close"].rolling(30).mean().iloc[-1]

        if not fund_row.empty:
            for col in ['PER', 'Yield', 'Capitalisation_flottante']:
                val = fund_row[col].values[0]
                if pd.notna(val):
                    self.indicators[col.lower() if col != 'Capitalisation_flottante' else 'cap_flo'] = val

    def get_indicator(self, name):
        return self.indicators.get(name)

    def get_all_indicators(self):
        return self.indicators

    def __repr__(self):
        return f"<Stock {self.ticker}>"


import pandas as pd

class StockEvaluator:
    def __init__(self, criteria=None):
        self.criteria = criteria or {
            "momentum": 0.3,
            "relative_price": 0.2,
            "volatility": 0.1,
            "volume": 0.15,
            "per": 0.1,
            "cap_flo": 0.15
        }

    def compute_score(self, stock):
        score = 0
        for key, weight in self.criteria.items():
            val = stock.get_indicator(key)
            if val is not None:
                score += weight * val
        return score

    def rank_stocks(self, stock_list):
        scored = [(stock, self.compute_score(stock)) for stock in stock_list]
        return sorted(scored, key=lambda x: x[1], reverse=True)

    def build_stocks_from_data(self, price_df: pd.DataFrame, fundamental_df: pd.DataFrame):
        grouped = price_df.groupby("Ticker")
        stocks = []
        for ticker, df in grouped:
            indicators = {}
            df = df.sort_values("Date")

            if len(df) >= 21:
                # Momentum = last close / close 20 days ago
                indicators["momentum"] = df["Close"].iloc[-1] / df["Close"].iloc[-21] - 1
            if len(df) >= 5:
                indicators["volatility"] = df["Close"].pct_change().rolling(5).std().iloc[-1]
                indicators["volume"] = df["Volume"].rolling(5).mean().iloc[-1]
                indicators["relative_price"] = df["Close"].iloc[-1] / df["Close"].rolling(30).mean().iloc[-1] if len(df) >= 30 else None

            fund_row = fundamental_df[fundamental_df['Ticker'] == ticker]
            if not fund_row.empty:
                for col in ['PER', 'Yield', 'Capitalisation_flottante']:
                    val = fund_row[col].values[0]
                    if pd.notna(val):
                        indicators[col.lower() if col != 'Capitalisation_flottante' else 'cap_flo'] = val

            stocks.append(Stock(ticker, indicators))
        return stocks


class Stock:
    def __init__(self, ticker, indicators):
        self.ticker = ticker
        self.indicators = indicators

    def get_indicator(self, name):
        return self.indicators.get(name)

    def get_all_indicators(self):
        return self.indicators

    def __repr__(self):
        return f"<Stock {self.ticker}>"


import pandas as pd

class MarketData:
    def __init__(self, df: pd.DataFrame):
        """
        df must have columns: ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']
        """
        self.df = df.copy()
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.df.set_index(['Date', 'Ticker'], inplace=True)

    def get_prices(self, kind='Close') -> pd.DataFrame:
        return self.df[kind].unstack()

    def get_volume(self) -> pd.DataFrame:
        return self.df['Volume'].unstack()

    def get_returns(self, kind='Close') -> pd.DataFrame:
        return self.get_prices(kind).pct_change()

    def get_ma(self, window=20) -> pd.DataFrame:
        return self.get_prices().rolling(window).mean()
    
    def clean(self, df: pd.DataFrame=None) -> pd.DataFrame:
        df = self.df if df is None else df
        return df.dropna(how='all').fillna(method='ffill')

    def normalize(self, df: pd.DataFrame=None) -> pd.DataFrame:
        df = self.df if df is None else df
        if "Description" in df:
            df = df.drop("Description", axis=1)
        return (df - df.mean()) / df.std()

    def fill_na(self, df: pd.DataFrame=None, method='ffill') -> pd.DataFrame:
        df = self.df if df is None else df
        return df.fillna(method=method)

    def reshape_to_matrix(self, df: pd.DataFrame=None, column: str="Close") -> pd.DataFrame:
        df = self.df if df is None else df
        return df.pivot_table(index='Date', columns='Ticker', values=column)

    
class FundamentalData:
    def __init__(self, fundamentals_df=None):
        """
        Must contain columns: ['ticker', 'PER', 'PB', 'ROE', 'NetMargin', 'Growth']
        """
        self.df = fundamentals_df.set_index('Ticker') if fundamentals_df is not None else self.get_fundamentals()
        
        
    def get_fundamentals(self):
        df = pd.read_csv("../DataBase/BRVM Stocks_Watchlist_06102025.csv")
        df.index, df.index.name = [e.split(".")[0] for e in df["Symbol"]], "Ticker"
        df["Date"] = f"{datetime.today().year}-{df['Time'][0].split('/')[1]}-{df['Time'][0].split('/')[0]}"
        df = df.drop(["Name", "Symbol", "Exchange", 'Bid', 'Ask',
                     'Extended Hours', 'Extended Hours (%)', '5 Minutes',
                     '15 Minutes','30 Minutes', 'Hourly', '5 Hours', 'Time',
                     'Last', 'Open', 'Prev.', 'High', 'Low', 'Chg.', 'Chg. %'], axis=1)
        df = (df
                 .replace("Strong Sell", -1)
                 .replace("Strong Buy", 1)
                 .replace("Neutral", 0)
                 .replace("Sell", -0.5)
                 .replace("Buy", 0.5)
                )
        # df["Ticker"] = list(df.index)
        df.columns = ['Volume', 'Next Earnings Date', 'Market Cap', 'Revenue',
       'Average 3m Volume', 'EPS', 'P/E Ratio', 'Beta', 'Dividend', 'Yield',
       'Daily Trend', 'Weekly Trend', 'Monthly Trend', '1 Day perf', '1 Week perf', '1 Month perf', 'YTD',
       '1 Year perf', '3 Years perf', 'Date']
        return df.reset_index(inplace=False)
        

    def get_per(self):
        return self.df['PER']

    def get_pb(self):
        return self.df['PB']

    def get_roe(self):
        return self.df['ROE']

    def get_margin(self):
        return self.df['NetMargin']

    def get_growth(self):
        return self.df['Growth']

    
class QuantitativeIndicators:
    def __init__(self, prices: pd.DataFrame, returns: pd.DataFrame, volume: pd.DataFrame):
        self.prices = prices
        self.returns = returns
        self.volume = volume

    def compute_momentum(self, window=20):
        return self.prices / self.prices.shift(window) - 1

    def compute_volatility(self, window=20):
        return self.returns.rolling(window).std()

    def compute_avg_volume(self, window=5):
        return self.volume.rolling(window).mean()

    def compute_relative_price(self, ma_window=20):
        return self.prices / self.prices.rolling(ma_window).mean() - 1

    
class Stock:
    def __init__(self, ticker: str, prices: pd.Series, returns: pd.Series = None):
        self.ticker = ticker
        self.prices = prices
        self.returns = returns if returns is not None else prices.pct_change()
        self.scores = {}

    def update_score(self, score_name: str, value: float):
        self.scores[score_name] = value

    def get_metric(self, metric: str):
        return self.scores.get(metric, None)


fd = FundamentalData()
kl_db._connect()
fd.df.to_sql("fundamental_data", kl_db.conn, if_exists="replace", index=False)
kl_db.close()